In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of flox._src.flow.api failed: Traceback (most recent call last):
  File "/home/jonas/miniconda3/envs/rigid-flows/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/jonas/miniconda3/envs/rigid-flows/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/home/jonas/miniconda3/envs/rigid-flows/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/home/jonas/miniconda3/envs/rigid-flows/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 345, in update_class
    update_instances(old, new)
  File "/home/jonas/miniconda3/envs/rigid-flows/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 303, in update_instances
    ref.__class__ = new
  File "/home/jonas/miniconda3/envs/rigid-flows/lib/python3.10/site-packages/jax_

In [25]:
import jax
import jax.numpy as jnp

import flox
from flox.util import key_chain
from flox import geom

from ribflox.rep import State, RigidRepresentation, RigidTransform, AtomRepresentation

seed = 42
chain = key_chain(seed)

ROT_DIM = 4
POS_DIM = 3
AUX_DIM = 5

NUM_MOL = 1

s = State(
    jax.vmap(RigidRepresentation)(
        rot=jax.vmap(geom.unit)(jax.random.normal(next(chain), shape=(NUM_MOL, ROT_DIM,))),
        pos=jax.random.uniform(next(chain), shape=(NUM_MOL, POS_DIM,))
    ),
    aux=jax.random.normal(next(chain), shape=(NUM_MOL, AUX_DIM)),
    box=jnp.ones(3)
)

In [26]:
from functools import partial
from ribflox.flow import full_step, WrappedLens
from lenses import lens
from flox.flow import Transform, Stack
import equinox as eqx

LATENT = 32
HIDDEN = (128,)
CTRL_PTS = 13

NUM_LAYERS = 3

def initial_transform() -> Transform[State[AtomRepresentation], State[RigidRepresentation]]:
    return flox.flow.LensedTransform(
        RigidTransform(),
        WrappedLens(lens.mol),
        WrappedLens(lens.mol)
    )


make_layer = jax.tree_util.Partial(full_step, num_mol=NUM_MOL, num_auxiliaries=AUX_DIM, num_latent=LATENT, num_hidden=HIDDEN, num_ctrl_pts=CTRL_PTS)
stack = eqx.filter_vmap(make_layer)(
    jax.random.split(next(chain), NUM_LAYERS))

flow = flox.flow.Pipe[State[AtomRepresentation], State[RigidRepresentation]]([
    initial_transform(),
    Stack(stack)
    # make_layer(next(chain))
])

In [27]:
from dataclasses import astuple

def unpack(obj):
    assert hasattr(obj, "__dataclass_fields__"), f"{obj} is not a dataclass"
    for field in obj.__dataclass_fields__:
        yield getattr(obj, field)


# s = State(
#     mol=jax.random.normal(next(chain), (NUM_MOL, 4, 3,)),
#     aux=jax.random.normal(next(chain), shape=(NUM_MOL, AUX_DIM)),
#     box=jnp.ones(3)
# )

@jax.jit
def foo(s):
    s_, _ = unpack(flow.inverse(s))
    # s_, _ = unpack(flow.forward(s))
    return s_

In [28]:
s_, _ = unpack(jax.jit(flow.inverse)(s))

In [32]:
%timeit foo(s)

461 µs ± 86.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
